# Image Processing

## Initial Plotly 2D Gray-Scale Histogram

In [4]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

def heatmap(directory, csv_name, x_branch, y_branch, title, x_label, y_label, width, height, size, bins, save_as):

    csv_file = os.path.join(directory, csv_name)
    df = pd.read_csv(csv_file)

    x_data = df[x_branch]
    y_data = df[y_branch]

    fig = go.Figure(go.Histogram2d(
                    x = x_data, 
                    y = y_data,
                    autobinx = False,
                    xbins = dict(start = -size, end = size, size = bins),
                    autobiny = False,
                    ybins = dict(start = -size, end = size, size = bins),
                    colorscale=[[0, 'white'], [1, 'black']],  # Grayscale from black to white
                    colorbar = dict(title = "Density", tickfont = dict(family = 'Merriweather', size = 16, color = 'Black'))))
    
    font_family = 'Merriweather'
    font_small  = 16
    font_medium = 20
    font_large  = 18
    
    fig.update_layout(
                    title = dict(text = title, font = dict(family = font_family, size = font_large, color = "Black"), 
                                 x = 0.51, y = 0.93, yanchor = 'middle', xanchor = 'center'),
                    xaxis_title = dict(text = x_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    yaxis_title = dict(text = y_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    xaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 25),
                    yaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 10),
                    width  = width,
                    height = height,
                    margin = dict(l = 105, r = 90, t = 90, b = 90),
                    )
    
    pio.write_image(fig, save_as, width = width, height = height, scale = 5)
    # fig.show()

In [7]:
directory = 'build/root/'
csv_name = 'aAF_30.csv'

x_branch = "X_axis"
y_branch = 'Y_axis'

title = r"$ \large{ \text{Osteoporosis.}} \ \normalsize{ \text{D to Target} =50cm, \ \text{D to Detector} =15cm, \ \theta = 0°} $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

size = 100
bins = 1

save_as = 'Results/aAF.jpg'

heatmap(directory, csv_name, x_branch, y_branch, title, x_label, y_label, width, height, size, bins, save_as)

## Root to Dataframe

In [ ]:
import uproot
import os
import csv
import numpy as np

def extract_root(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, decimal_places):
    
    file_name = os.path.join(directory, root_name_starts)

    if file_name.endswith('.root') and file_name.startswith(root_name_starts):
        file_path = os.path.join(directory, file_name)

        try:
            with uproot.open(file_path) as root_file:
                tree = root_file[tree_name]
                if tree is None and count == 0:
                    print(f"Tree '{tree_name}' not found in {file_path}")
                    count += 1

                # Extract branches as arrays
                x_values = tree[x_branch].array() if x_branch in tree else None
                y_values = tree[y_branch].array() if y_branch in tree else None
                z_values = tree[z_branch].array() if z_branch in tree else None

                x_values = np.round(x_values, decimal_places) if x_branch in tree else None
                y_values = np.round(y_values, decimal_places) if x_branch in tree else None



        except Exception as e: print(f"Error processing {file_path}: {e}")

    return 

In [8]:
import uproot
import pandas as pd
import os
import numpy as np

def extract_root(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, decimal_places):
    
    file = os.path.join(directory, root_name_starts + '.root')

    try:
        with uproot.open(file) as root_file:
            tree = root_file[tree_name]
            if tree is None:
                print(f"Tree '{tree_name}' not found in {file}")
                return None
            
            # Extract branches as arrays
            x_values = tree[x_branch].array(library="np") if x_branch in tree else None
            y_values = tree[y_branch].array(library="np") if y_branch in tree else None
            z_values = tree[z_branch].array(library="np") if z_branch in tree else None

            # Round values to specified decimal places
            if x_values is not None:
                x_values = np.round(x_values, decimal_places)
            if y_values is not None:
                y_values = np.round(y_values, decimal_places)
            if z_values is not None:
                z_values = np.round(z_values, decimal_places)

            # Create a DataFrame with the extracted values
            data = {
                x_branch: x_values,
                y_branch: y_values,
                z_branch: z_values
            }

            df = pd.DataFrame(data)
            
            return df

    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

In [1]:
import uproot
import pandas as pd
import os
import numpy as np

def extract_root(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, decimal_places, chunk_size=10000):
    
    file = os.path.join(directory, root_name_starts + '.root')

    try:
        with uproot.open(file) as root_file:
            tree = root_file[tree_name]
            if tree is None:
                print(f"Tree '{tree_name}' not found in {file}")
                return None
                
                # Create an empty DataFrame to accumulate the results
            df = pd.DataFrame()

            # Iterate over chunks of the data
            for start in range(0, tree.num_entries, chunk_size):
                end = min(start + chunk_size, tree.num_entries)
                
                # Extract branches as arrays for the current chunk
                x_values = tree[x_branch].array(library="np", entry_start=start, entry_stop=end) if x_branch in tree else None
                y_values = tree[y_branch].array(library="np", entry_start=start, entry_stop=end) if y_branch in tree else None
                z_values = tree[z_branch].array(library="np", entry_start=start, entry_stop=end) if z_branch in tree else None

                # Round values to specified decimal places
                if x_values is not None:
                    x_values = np.round(x_values, decimal_places)
                if y_values is not None:
                    y_values = np.round(y_values, decimal_places)
                if z_values is not None:
                    z_values = np.round(z_values, decimal_places)

                # Create a DataFrame for the current chunk
                chunk_df = pd.DataFrame({
                    x_branch: x_values,
                    y_branch: y_values,
                    z_branch: z_values
                })

                # Append chunk to the overall DataFrame
                df = pd.concat([df, chunk_df], ignore_index=True)
                
            return df

    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

In [ ]:
directory = 'build/root/'
root_name_starts = "merge0"

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"
z_branch  = ""

decimal_places = 2

dataframe = extract_root(directory, root_name_starts, tree_name, x_branch, y_branch, z_branch, decimal_places)

## Numpy Heatmap Matrix, Normalize

In [66]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def heatmap_array(directory, csv_name, x_branch, y_branch, size, num):
    
    csv_file = os.path.join(directory, csv_name)
    df = pd.read_csv(csv_file)

    x_data = df[x_branch]
    y_data = df[y_branch]

    set_bins = np.arange(-size, size + 1, size/num)
    heatmap, x_edges, y_edges = np.histogram2d(x_data, y_data, bins = [set_bins, set_bins])
    heatmap = heatmap.T
    # print(heatmap.shape)

    row = len(set_bins) // 2
    normal_map = 1 - heatmap / np.max(heatmap)
    plt.plot(normal_map[:][row])
    maxi = np.max(normal_map[:5])
    maxi = maxi * 1.2
    print('altura de ruido:', round(maxi, 4))
    normal_map[normal_map < maxi] = 0

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(normal_map, cmap='gray', extent = [x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    plt.subplot(1, 2, 2)
    plt.plot(normal_map[:][row])
    # plt.savefig('Results/circ1_2.jpg', dpi = 300)

    return normal_map, x_edges, y_edges

    # optional: save array as csv
    # np.savetxt("Results/array2.csv", htmp_array, delimiter=",", fmt="%f")
    # print(htmp_array.shape)

In [ ]:
directory = 'build/root/'
csv_name = 'jp1.csv'

x_branch = "X_axis"
y_branch = 'Y_axis'

size = 100
bins = 60

htmp_array, xlim, ylim = heatmap_array(directory, csv_name, x_branch, y_branch, size, bins)

## Eliminate Noise by Fourier Transform

In [12]:
import numpy as np
from scipy.fft import fft2, fftshift, ifft2
import matplotlib.pyplot as plt
from scipy import signal

def Denoise(htmp_array, hann, alpha):

    image = htmp_array

    fft_image = fft2(image)
    fft_image = fftshift(fft_image)

    rows, cols = image.shape

    hann = False
    if hann == True:
    
        l = rows * alpha
        a = np.hanning(l)
        b = np.hanning(l)

        padding_size = rows - len(a)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        a = np.pad(a, (left_padding, right_padding), mode='constant')

        padding_size = cols - len(b)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        b = np.pad(b, (left_padding, right_padding), mode='constant')

        window = np.outer(a, b)

    else:

        a = signal.windows.tukey(rows, alpha)
        b = signal.windows.tukey(rows, alpha)
        window = np.outer(a, b)

    plt.figure(figsize=(10, 2))
    plt.subplot(1, 3, 2)
    plt.plot(a)

    fft_image_2 = fft_image * (window)

    plt.subplot(1, 3, 3)
    plt.plot(np.abs((fft_image_2[:][rows//2])))

    fft_image = fftshift(fft_image_2)
    fft_image = (ifft2(fft_image))
    fft_image = (np.abs(fft_image))

    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title('Filtered Image')
    plt.imshow(fft_image, cmap='gray')
    plt.savefig('Results/four1.jpg', dpi = 300)
    plt.show()

    print(fft_image.shape)
    plt.plot(fft_image[:][60])

    return fft_image

In [ ]:
hann = False
alpha = .1
htmp_array = dataframe
fft_image = Denoise(htmp_array, hann, alpha)

## Save New Heatmap from Numpy Array

In [23]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

def heatmap2(array, xlim, ylim, title, x_label, y_label, width, height, save_as):

    fig = go.Figure(go.Heatmap(
                    z = array,
                    x = xlim,
                    y = ylim,
                    colorscale = [[0, 'black'], [1, 'white']],  # Grayscale from black to white
                    colorbar = dict(title = "Density", tickfont = dict(family = 'Merriweather', size = 16, color = 'Black'))))
    
    font_family = 'Merriweather'
    font_small  = 16
    font_medium = 20
    font_large  = 18
    
    fig.update_layout(
                    title = dict(text = title, font = dict(family = font_family, size = font_large, color = "Black"), 
                                 x = 0.51, y = 0.93, yanchor = 'middle', xanchor = 'center'),
                    xaxis_title = dict(text = x_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    yaxis_title = dict(text = y_label, font = dict(family = font_family, size = font_medium, color = "Black")),
                    xaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 25),
                    yaxis = dict(tickfont = dict(family = font_family, size = font_small, color = "Black"), title_standoff = 10),
                    width  = width,
                    height = height,
                    margin = dict(l = 105, r = 90, t = 90, b = 90)
    )
    
    pio.write_image(fig, save_as, width = width, height = height, scale = 5)
    fig.show()

In [ ]:
array = fft_image
# array = htmp_array

title = r"$ \large{ \text{Osteoporosis.}} \ \normalsize{ \text{D to Target} =50cm, \ \text{D to Detector} =15cm, \ \theta = 0°} $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

save_as = 'Results/test.jpg'

heatmap2(array, xlim, ylim, title, x_label, y_label, width, height, save_as)